In [1]:
%load_ext gams_magic

# Questions: 

1. Now, suppose that the demand of city 1 reduces to 40, and any unused supply causes a holding cost of $\$$ 2 per million kwh. How to minimize the total cost in this new setting? (Optimal value: $\$$ 985)

<br>

2. Next, let the demand of city 1 be 45 as before, but suppose that the supply of plant 1 reduces to 30 from the original amount of 35. Suppose that any unmet demands are penalized by a charge of $\$$ 20 per million kwh at city 1, $\$$ 30 per million kwh at city 2, $\$$ 30 per million kwh at city 3, and $\$$ 40 per million kwh at city 4. How to minimize the total cost in this new setting? (Optimal value: $\$$ 1090)

In [2]:
%%gams

#Solution to the original problem

# set declaration

Sets
       i   "plants"   / p1*p3 /,
       j   "cities"   / c1*c4 / ;

# scalars and parameters
Parameters
    s(i)  "supply capacity of plant i"
         /  p1 35
            p2 50
            p3 40 /

    d(j)  "demand at city j"
         /  c1    45
            c2    20
            c3    30
            c4    30  / ;

table c(i,j)  "unit shipping cost"
              c1       c2      c3      c4
    p1        8        6       10      9
    p2        9        12      13      7
    p3        14       9       16      5;

#also correct:
# parameters c(i,j) /p1.c1 8, p1.c2 6,..../;

# variable declaration
Variables
    x(i,j)
    z       "total transportation costs";

Positive Variable x ;

#equation declaration
Equations
    cost        define objective function
    supply(i)   observe supply limit at plant i
    demand(j)   satisfy demand at city j ;

#equation definition
cost ..   z  =e=  sum((i,j), c(i,j)*x(i,j)) ;
# z=e= c('p1','c1')* x('p1','c1')+ 6*x('p1','c2') +....;

supply(i) ..   sum(j, x(i,j))  =e=  s(i) ;

demand(j) ..   sum(i, x(i,j))  =e=  d(j) ;

# model and solve
Model transport /all/ ;
  
#GAMS option limrow specifies the number of equations to include in GAMS listing file
option limrow = 3;

#GAMS option solprint on/off: whether to not to include the solution report part in the listing file
option solprint = on;

Solve transport using lp minimizing z ;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1020.0,8,13,LP,OSICPLEX,0


In [3]:
%gams display x.l, z.l;
%gams_lst -e

E x e c u t i o n


----     67 VARIABLE x.L  

            c1          c2          c3          c4

p1                  10.000      25.000
p2      45.000                   5.000
p3                  10.000                  30.000


----     67 VARIABLE z.L                   =     1020.000  total transportation costs




In [4]:
%gams_reset
#With a %gams_reset we can reset the GAMS database and can declare symbols with a different type and domain/dimension.

In [5]:
%%gams

# Solution 1 for question 1

# set declaration

Sets
       i   "plants"   / p1*p3 /
       j   "cities"   / c1*c4, dummy/ ;

# scalars and parameters
Parameters
    s(i)  "supply capacity of plant i"
         /  p1 35
            p2 50
            p3 40 /

    d(j)  "demand at city j"
         /  c1    40
            c2    20
            c3    30
            c4    30 
            dummy 5/ ;

table c(i,j)  "unit shipping cost"
              c1       c2      c3      c4   dummy
    p1        8        6       10      9     2
    p2        9        12      13      7     2
    p3        14       9       16      5     2;

#also correct:
# parameters c(i,j) /p1.c1 8, p1.c2 6,..../;

# variable declaration
Variables
    x(i,j)
    z       "total transportation costs";

Positive Variable x ;

#equation declaration
Equations
    cost        define objective function
    supply(i)   observe supply limit at plant i
    demand(j)   satisfy demand at city j ;

#equation definition
cost ..   z  =e=  sum((i,j), c(i,j)*x(i,j)) ;

supply(i) ..   sum(j, x(i,j))  =e=  s(i) ;

demand(j) ..   sum(i, x(i,j))  =e=  d(j) ;

# model and solve
Model transport /all/ ;
  
#GAMS option limrow specifies the number of equations to include in GAMS listing file
option limrow = 3;

#GAMS option solprint on/off: whether to not to include the solution report part in the listing file
option solprint = on;

Solve transport using lp minimizing z ;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),985.0,9,16,LP,OSICPLEX,0.001


In [6]:
%gams_reset

In [7]:
%%gams

#Solution 2 of question 1

# set declaration

Sets
       i   "plants"   / p1*p3 /
       j   "cities"   / c1*c4 / ;

# scalars and parameters
Parameters
    s(i)  "supply capacity of plant i"
         /  p1 35
            p2 50
            p3 40 /

    d(j)  "demand at city j"
         /  c1    40
            c2    20
            c3    30
            c4    30  / ;

table c(i,j)  "unit shipping cost"
              c1       c2      c3      c4
    p1        8        6       10      9
    p2        9        12      13      7
    p3        14       9       16      5;

#also correct:
# parameters c(i,j) /p1.c1 8, p1.c2 6,..../;

# variable declaration
Variables
    x(i,j)
    z       "total transportation costs";

Positive Variable x ;

#equation declaration
Equations
    cost        define objective function
    supply(i)   observe supply limit at plant i
    demand(j)   satisfy demand at city j ;

#equation definition
cost ..   z  =e=  sum((i,j), c(i,j)*x(i,j)) + sum(i, 2*(s(i)-sum(j,x(i,j))));

supply(i) ..   sum(j, x(i,j))  =l=  s(i) ;

demand(j) ..   sum(i, x(i,j))  =g=  d(j) ;

# model and solve
Model transport /all/ ;
  
#GAMS option limrow specifies the number of equations to include in GAMS listing file
option limrow = 3;

#GAMS option solprint on/off: whether to not to include the solution report part in the listing file
option solprint = on;

Solve transport using lp minimizing z ;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),985.0,8,13,LP,OSICPLEX,0


In [8]:
%gams_reset

In [9]:
%%gams

#Solution 1 to question 2

# set declaration

Sets
       i   "plants"   / p1*p3,dummy /
       j   "cities"   / c1*c4 / ;

# scalars and parameters
Parameters
    s(i)  "supply capacity of plant i"
         /  
            p1 30
            p2 50
            p3 40
            dummy 5/

    d(j)  "demand at city j"
         /  c1    45
            c2    20
            c3    30
            c4    30  / ;

table c(i,j)  "unit shipping cost"
              c1       c2      c3      c4
    p1        8        6       10      9
    p2        9        12      13      7
    p3        14       9       16      5
    dummy     20       30      30      40;

#also correct:
# parameters c(i,j) /p1.c1 8, p1.c2 6,..../;

# variable declaration
Variables
    x(i,j)
    z       "total transportation costs";

Positive Variable x ;

#equation declaration
Equations
    cost        define objective function
    supply(i)   observe supply limit at plant i
    demand(j)   satisfy demand at city j ;

#equation definition
cost ..   z  =e=  sum((i,j), c(i,j)*x(i,j)) ;

supply(i) ..   sum(j, x(i,j))  =e=  s(i) ;

demand(j) ..   sum(i, x(i,j))  =e=  d(j) ;

# model and solve
Model transport /all/ ;
  
#GAMS option limrow specifies the number of equations to include in GAMS listing file
option limrow = 3;

#GAMS option solprint on/off: whether to not to include the solution report part in the listing file
option solprint = on;

Solve transport using lp minimizing z ;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1090.0,9,17,LP,OSICPLEX,0


In [10]:
%gams_reset

In [11]:
%%gams

#solution 2 to question 2

# set declaration

Sets
       i   "plants"   / p1*p3 /
       j   "cities"   / c1*c4 / ;

# scalars and parameters
Parameters
    s(i)  "supply capacity of plant i"
         /  p1 30
            p2 50
            p3 40 /

    d(j)  "demand at city j"
         /  c1    45
            c2    20
            c3    30
            c4    30  /
            
    penalty(j) / c1 20, c2 30, c3 30, c4 40/
    ;

table c(i,j)  "unit shipping cost"
              c1       c2      c3      c4
    p1        8        6       10      9
    p2        9        12      13      7
    p3        14       9       16      5;

#also correct:
# parameters c(i,j) /p1.c1 8, p1.c2 6,..../;

# variable declaration
Variables
    x(i,j)
    z       "total transportation costs";

Positive Variable x ;

#equation declaration
Equations
    cost        define objective function
    supply(i)   observe supply limit at plant i
    demand(j)   satisfy demand at city j ;

#equation definition
cost ..   z  =e=  sum((i,j), c(i,j)*x(i,j)) +sum(j, penalty(j)*(d(j)-sum(i,x(i,j))));

supply(i) ..   sum(j, x(i,j))  =l=  s(i) ;

demand(j) ..   sum(i, x(i,j))  =l=  d(j) ;

# model and solve
Model transport /all/ ;
#Wrong: Model transport /cost, supply/ ;
  
#GAMS option limrow specifies the number of equations to include in GAMS listing file
option limrow = 3;

#GAMS option solprint on/off: whether to not to include the solution report part in the listing file
option solprint = on;

Solve transport using lp minimizing z ;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1090.0,8,13,LP,OSICPLEX,0


In [12]:
%gams_cleanup